In [ ]:
!pip install timm
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import PIL
from torch import nn
from torchvision import models
from torch.utils.data import DataLoader, Dataset
import timm
import time
from tqdm import tqdm
from torch.cuda.amp import autocast, GradScaler
import random
import torch

import sklearn
from sklearn.model_selection import StratifiedKFold 


from albumentations import (
    HorizontalFlip, VerticalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine, RandomResizedCrop,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose, Normalize, Cutout, CoarseDropout, ShiftScaleRotate, CenterCrop, Resize
)
from albumentations.pytorch import ToTensorV2

%matplotlib inline

In [ ]:
Base_dir = '../input/siim-isic-melanoma-classification/'
df_train = pd.read_csv(os.path.join(Base_dir,'train.csv'))
df_train.head(2)

In [ ]:
df_train = pd.read_csv(os.path.join(Base_dir,'train.csv'))
df_train.head(2)

In [ ]:
import PIL


In [ ]:
'''
    Helper Functions
'''

def get_img(path):
    im_bgr = cv2.imread(path)
    im_rgb = cv2.cvtColor(im_bgr,cv2.COLOR_BGR2RGB)
    return im_rgb

def seed_everything(seed):
    '''
        creates seed for the environment
    '''
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

In [ ]:
CFG = {
    'fold_num' : 5,
    'seed' : 48,
    'model_arch' : 'tf_efficientnet_b4_ns',
    'img_size':  512,
    'epochs' : 10,
    'train_bs' : 32,
    'valid_bs': 16,
    'lr': 1e-4,
    'min_lr': 1e-6,
    'weight_decay':1e-6,
    'num_workers': 4,
    'accum_iter': 2, # suppoprt to do batch accumulation for backprop with effectively larger batch size
    'verbose_step': 1,
    'device': 'cpu',
    'T_0':10,
    'pretrained':'imagenet'
}

In [ ]:
plt.figure(figsize=(10,10))
sns.countplot(df_train.target,data=df_train)

In [ ]:
class ImageDataset(Dataset):
    
    def __init__(self, df, data_root, transforms=None, output_labels=None):
        super().__init__()
        self.df = df.reset_index(drop=True).copy()
        self.data_root = data_root
        self.transforms = transforms
        self.output_labels = output_labels

    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, index:int):
        if self.output_labels:
            target = self.df['target'][index]
        
        img = get_img(os.path.join((self.data_root),f'{self.df["image_name"][index]}.jpg'))
        
        if (self.transforms):
            img = self.transforms(image=img)['image']
        
        if self.output_labels:
            return img, target
        else:
            return img

In [ ]:
def get_train_aug():
    Compose([
        RandomResizedCrop(CFG['img_size'],CFG['img_size']),
        Transpose(p=0.5),
        VerticalFlip(p=0.5),
        HorizontalFlip(p=0.5),
        Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.255],max_pixel_value=255.0,p=1.0),
        ToTensorV2(p=1.0)
    ],p=1.0)
    
def get_valid_aug():
    Compose([
        CenterCrop(CFG['img_size'],CFG['img_size'],p=1.0),
        Resize(CFG['img_size'],CFG['img_size']),
        Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.255],max_pixel_value=255.0,p=1.0),
        ToTensorV2(p=1.0)
    ],p=1.0)

In [ ]:
class ImageClassifier(nn.Module):
    def __init__(self, model_arch, n_class, pretrained=False):
        super().__init__()
        self.model = timm.create_model(model_arch,pretrained=pretrained)
        n_features = self.model.classifier.in_features
        self.model.classifier = nn.Linear(n_features, n_class)
    def forward(self, x):
        x = self.model(x)
        return x

In [ ]:
def prepare_dataloader(df, trn_idx, val_idx, data_root='../input/siim-isic-melanoma-classification/jpeg/train/'):
    train_ = df.loc[trn_idx,:].reset_index(drop=True)
    valid_ = df.loc[val_idx,:].reset_index(drop=True)
    
    train_ds = ImageDataset(train_, data_root, transforms = get_train_aug(), output_labels=True)
    valid_ds = ImageDataset(valid_, data_root, transforms = get_valid_aug(), output_labels=True)
    
    train_loader = DataLoader(
        train_ds,
        batch_size = CFG['train_bs'],
        pin_memory = False,
        drop_last = False,
        shuffle = True,
        num_workers = CFG['num_workers'],
    )
    
    valid_loader = DataLoader(
        valid_ds,
        batch_size = CFG['valid_bs'],
        pin_memory = False,
        shuffle = False,
        num_workers = CFG['num_workers'],
    )
    
    return train_loader, valid_loader



In [ ]:
def train_one_epoch(epoch, model, loss_fn, optimizer, train_loader, device, scheduler=None, schd_batch_update=False):
    model.train()
    
    t = time.time()
    running_loss = None
    
    pbar = tqdm(enumerate(train_loader), total = len(train_loader))
    for step, (img, img_target) in pbar:
        img = img.to_device(device).float()
        img_target = img_target.to_device(device).long()
        
        with autocast():
            
            img_preds = model(img)
            loss = loss_function(img_preds, img_targets)
            scaler.scale(loss).backward()
            
            if running_loss == None:
                running_loss = loss.item()
            else:
                running_loss = 0.99*running_loss + 0.01*loss.item()
            
            if ((step+1) % CFG['accum_iter'] == 0) or ((step+1) == len(train_loader)):
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()
                
            if scheduler is not None and not schd_batch_update:
                scheduler.step()
                
            if ((step+1) % CFG['verbose_step'] == 0) or ((step+1) == len(train_loader)):
                
                description = f'epoch: {epoch}, loss: {running_loss}'
                pbar.set_description(description)
                
                
    if scheduler is not None and schd_batch_update:
        scheduler.step()
        
def valid_one_epoch(epoch, model, loss_fn, valid_loader, device, sceduler=None, schd_loss_update=False):
    model.eval()
    
    t = time.time()
    loss_sum = 0
    sample_num = 0
    img_preds_all = []
    img_targets_all = []
    
    pbar = tqdm(enumerate(valid_loader), total = len(valid_loader))
    for step, (img, img_target) in pbar:
        img = img.to(device).float()
        img_target = img_target.to(device).long()
        
        img_preds = model(img)
        img_preds_all += [img_preds.detach().cpu().numpy()]
        img_targets_all += [img_targets.detach().cpu().numpy()]
        
        loss = loss_fn(img_preds, img_target)
        
        loss_sum += loss.item()*img_target.shape[0]
        sample_num += img_target.shape[0]
        
        if ((step+1) % CFG['verbose_step'] == 0) or ((step+1) == len(valid_loader)):
            description = f'epochs: {epoch}, loss: {loss_num/sample_num:.4}'
            pbar.set_description(description)
    
    img_preds_all = np.concatenate(img_preds_all)
    img_targets_all = np.concatenate(img_targets_all)
    print(f'validation binary accuracy: {(img_preds_all == img_targets_all).mean():.4}')
    
    if scheduler is not None:
        if schd_loss_update:
            scheduler.step(loss_sum/sample_num)
        else:
            scheduler.step()

In [ ]:
if __name__ == "__main__":
    
    seed_everything(CFG['seed'])
    
    folds = StratifiedKFold(n_splits = CFG['fold_num']).split(np.arange(df_train.shape[0]), df_train.target.values)
    
    test_preds = None,
    
    for fold, (trn_idx, val_idx) in enumerate(folds):
        
        print(f'Training with fold {fold} started')
        print(len(trn_idx), len(val_idx))
        
        train_loader, val_loader = prepare_dataloader(df_train, trn_idx, val_idx, data_root='../input/siim-isic-melanoma-classification/jpeg/train/')
        device = torch.device(CFG['device'])
        model = ImageClassifier(CFG['model_arch'],df_train.target.nunique()-1,pretrained=True).to(device)
        scaler = GradScaler()
        optimizer = torch.optim.Adam(model.parameters(),lr=CFG['lr'], weight_decay=CFG['weight_decay'])
        scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer,T_0=CFG['T_0'],T_mult=1, eta_min=CFG['min_lr'], last_epoch=-1)
        
        loss_tr = nn.BCELoss().to(device)
        loss_fn = nn.BCELoss().to(device)
        
        for epoch in range(CFG['epochs']):
            train_one_epoch(epoch, model, loss_tr, optimizer, train_loader, device, scheduler=scheduler, schd_batch_update=False)
            with torch.no_grad():
                valid_one_epoch(epoch, model, loss_fn, val_loader, device, scheduler=None, schd_loss_update=False)
            
            test_preds += [model()]
        
        del model, optimizer, train_loader, val_loader, scaler, scheduler
        torch.cuda.empty_cache()